<a href="https://colab.research.google.com/github/Bluseass/Dataset/blob/main/sms_spam_with_view.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download() # d // all // q

In [5]:
import pandas as pd
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


data = pd.read_csv("/content/drive/MyDrive/nlp/SMSSpamCollection", sep='\t')
data.columns = ['label', 'body_text']

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

def clean_text(text):
    text = ''.join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = ' '.join([ps.stem(word) for word in tokens if word not in stopwords])
    return text

data['clean_text'] = data['body_text'].apply(lambda x: clean_text(x))


X_train, X_test, y_train, y_test = train_test_split(data['clean_text'], data['label'], test_size=0.2, random_state=42)


tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_train_tfidf = tfidf_vect.fit_transform(X_train)
X_test_tfidf = tfidf_vect.transform(X_test)


svm_model = SVC(kernel='linear')
svm_model.fit(X_train_tfidf, y_train)


predictions = svm_model.predict(X_test_tfidf)


print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test, predictions))

[[952   3]
 [ 21 139]]
              precision    recall  f1-score   support

         ham       0.98      1.00      0.99       955
        spam       0.98      0.87      0.92       160

    accuracy                           0.98      1115
   macro avg       0.98      0.93      0.95      1115
weighted avg       0.98      0.98      0.98      1115

0.97847533632287


ВЬЮШКА


In [6]:
import ipywidgets as widgets
def analyze_text(text):
    text_tfidf = tfidf_vect.transform([text])
    prediction = svm_model.predict(text_tfidf)
    return f"Результат анализа: {prediction[0]}"

# Функция обработки нажатия кнопки анализировать
def analyze_button_clicked(b):
    result = analyze_text(input_text.value)
    output_text.value = result

# Функция обработки нажатия кнопки очистить
def clear_button_clicked(b):
    input_text.value = ''
    output_text.value = ''

# Создание текстового поля для ввода текста
input_text = widgets.Textarea(description='Текст:', layout={'width': '400px', 'height': '200px'})

# Создание кнопки для запуска анализа текста
analyze_button = widgets.Button(description='Анализировать')
analyze_button.on_click(analyze_button_clicked)

# Создание кнопки для очистки поля ввода
clear_button = widgets.Button(description='Очистить')
clear_button.on_click(clear_button_clicked)

# Создание текстового поля для вывода результата анализа
output_text = widgets.Textarea(description='Результат:', layout={'width': '400px', 'height': '50px'})

# Отображение виджетов
display(input_text)
display(widgets.HBox([analyze_button, clear_button]))
display(output_text)

Textarea(value='', description='Текст:', layout=Layout(height='200px', width='400px'))

Textarea(value='', description='Результат:', layout=Layout(height='50px', width='400px'))